In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-the-number-of-upvotes-a-post-will-get/test_8i3B3FC.csv
/kaggle/input/predict-the-number-of-upvotes-a-post-will-get/train_NIR5Yl1.csv


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

In [4]:
train_df = pd.read_csv("/kaggle/input/predict-the-number-of-upvotes-a-post-will-get/train_NIR5Yl1.csv")
test_df = pd.read_csv("/kaggle/input/predict-the-number-of-upvotes-a-post-will-get/test_8i3B3FC.csv")

In [5]:
train_df.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [6]:
train_df.isna().sum()

ID            0
Tag           0
Reputation    0
Answers       0
Username      0
Views         0
Upvotes       0
dtype: int64

In [7]:
train_df.drop(['ID', 'Username'], axis=1, inplace=True)
train_df.head()

,Tag,Reputation,Answers,Views,Upvotes
0,a,3942.0,2.0,7855.0,42.0
1,a,26046.0,12.0,55801.0,1175.0
2,c,1358.0,4.0,8067.0,60.0
3,a,264.0,3.0,27064.0,9.0
4,c,4271.0,4.0,13986.0,83.0


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330045 entries, 0 to 330044
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Tag         330045 non-null  object 
 1   Reputation  330045 non-null  float64
 2   Answers     330045 non-null  float64
 3   Views       330045 non-null  float64
 4   Upvotes     330045 non-null  float64
dtypes: float64(4), object(1)
memory usage: 12.6+ MB


In [9]:
for label, content in train_df.items() :
    if pd.api.types.is_object_dtype(content) :
        train_df[label] = content.astype('category')

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330045 entries, 0 to 330044
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   Tag         330045 non-null  category
 1   Reputation  330045 non-null  float64 
 2   Answers     330045 non-null  float64 
 3   Views       330045 non-null  float64 
 4   Upvotes     330045 non-null  float64 
dtypes: category(1), float64(4)
memory usage: 10.4 MB


In [11]:
for label, content in train_df.items() :
    if pd.api.types.is_categorical_dtype(content) :
        train_df[label] = content.cat.codes+1

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330045 entries, 0 to 330044
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Tag         330045 non-null  int8   
 1   Reputation  330045 non-null  float64
 2   Answers     330045 non-null  float64
 3   Views       330045 non-null  float64
 4   Upvotes     330045 non-null  float64
dtypes: float64(4), int8(1)
memory usage: 10.4 MB


In [13]:
X = train_df.drop('Upvotes', axis=1)
y = train_df.Upvotes

In [14]:
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((231031, 4), (231031,), (99014, 4), (99014,))

In [15]:
models = {"ada_reg" : AdaBoostRegressor(),
          "gra_reg" : GradientBoostingRegressor()}

In [19]:
def fit_and_score (models, X_train, y_train, X_test, y_test) :
    models_score = {}
    model_preds = {}
    for name, model in models.items() :
        model.fit(X_train, y_train)
        models_score[name] = model.score(X_test, y_test)
    return models_score

In [20]:
%%time

models_Scores = fit_and_score(models, X_train, y_train, X_test, y_test)
models_Scores

CPU times: user 32.8 s, sys: 435 ms, total: 33.3 s
Wall time: 33.3 s


{'ada_reg': 0.5110708905763022, 'gra_reg': 0.9170968551400526}

In [21]:
gra_reg =  GradientBoostingRegressor()

In [22]:
gra_reg.fit(X_train, y_train)

GradientBoostingRegressor()

In [23]:
preds = gra_reg.predict(X_test)
mean_absolute_error(y_test, preds)

150.230161069347

In [24]:
test_df.drop(['ID', 'Username'], axis=1, inplace=True)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141448 entries, 0 to 141447
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Tag         141448 non-null  object 
 1   Reputation  141448 non-null  float64
 2   Answers     141448 non-null  float64
 3   Views       141448 non-null  float64
dtypes: float64(3), object(1)
memory usage: 4.3+ MB


In [25]:
for label, content in test_df.items() :
    if pd.api.types.is_object_dtype(content) :
        test_df[label] = content.astype('category')

In [26]:
for label, content in test_df.items() :
    if pd.api.types.is_categorical_dtype(content) :
        test_df[label] = content.cat.codes+1

In [27]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141448 entries, 0 to 141447
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Tag         141448 non-null  int8   
 1   Reputation  141448 non-null  float64
 2   Answers     141448 non-null  float64
 3   Views       141448 non-null  float64
dtypes: float64(3), int8(1)
memory usage: 3.4 MB


In [28]:
test_preds = gra_reg.predict(test_df)
final_result = pd.DataFrame(test_preds)
final_result.head()

,0
0,210.160413
1,339.794884
2,42.616609
3,42.616609
4,308.131437


In [29]:
final_result.columns = ['Upvotes']

In [30]:
final_result.head()

,Upvotes
0,210.160413
1,339.794884
2,42.616609
3,42.616609
4,308.131437
